In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-data/movie_data.csv


In [2]:
df=pd.read_csv("/kaggle/input/movie-data/movie_data.csv")

In [3]:
df

,Unnamed: 0,Title,Year,Id,Overview,TMDB_Id,Average_vote,Popularity,Revenue,Budget,Genre,Keywords,Cast,Director,Poster
0,0,Mr Joe B. Carvalho,2014.0,tt2876408,A small time detective is hired to bring back ...,250726.0,3.100,1.832,NaN,NaN,['Comedy'],[],"['Arshad Warsi', 'Soha Ali Khan', 'Javed Jaffr...",NaN,https://media.themoviedb.org/t/p/w600_and_h900...
1,1,Sholay 3D,2014.0,tt0073707,After his family is murdered by a notorious an...,12259.0,6.979,16.705,4800000.0,408000.0,['Action'],"['police', 'fight', 'village', 'bandit']","['Amitabh Bachchan', 'Om Shivpuri', 'Amjad Khan']",Ramesh Sippy,https://media.themoviedb.org/t/p/w600_and_h900...
2,2,Dedh Ishqiya,2014.0,tt2675978,A team of con men fall for a Begum and her fem...,248698.0,6.500,11.377,NaN,NaN,"['Comedy', 'Drama', 'Romance']",[],"['Vijay Raaz', 'Naseeruddin Shah', 'Manoj Pahwa']",Vishal Bhardwaj,https://media.themoviedb.org/t/p/w600_and_h900...
3,3,Yaariyan,2014.0,tt3382148,A college student must win an inter college co...,252053.0,3.222,3.705,100.0,10.0,"['Drama', 'Romance']","['competition', 'college']","['Rakul Preet Singh', 'Gulshan Grover', 'Evely...",NaN,https://media.themoviedb.org/t/p/w600_and_h900...
4,4,Karle Pyaar Karle,2014.0,tt3465488,"Karle Pyaar Karle is an adrenaline gushing, ac...",252051.0,0.000,3.592,NaN,NaN,"['Action', 'Romance']",[],"['Hasleen Kaur', 'Shiv Darshan', 'Aham Sharma']",Suneel Darshan,https://media.themoviedb.org/t/p/w600_and_h900...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,1005,Kaisi Ye Dor,2023.0,tt29666603,Set in the backdrop of the spiritual capital o...,1211406.0,0.000,2.73,NaN,2700000.0,"['Drama', 'Family']",['independent film'],"['Sunita Rajwar', 'Brijendra Kala', 'Jashn Agn...",Sandeep Choudhary,https://media.themoviedb.org/t/p/w600_and_h900...
1006,1006,Dunki,2023.0,tt15428134,Four friends from a sleepy little village in P...,960876.0,6.400,25.042,NaN,15000000.0,"['Adventure', 'Comedy', 'Drama']",[],"['Shah Rukh Khan', 'Taapsee Pannu', 'Vicky Kau...",Natasha Raj Mehta,https://media.themoviedb.org/t/p/w600_and_h900...
1007,1007,Dry Day,2023.0,tt28024755,When his wife makes a vow to abort their baby ...,1216709.0,7.500,12.222,NaN,NaN,"['Comedy', 'Drama']",[],"['Jitendra Kumar', 'Annu Kapoor', 'Shriya Pilg...",Nikkhil Advani,https://media.themoviedb.org/t/p/w600_and_h900...
1008,1008,Kho Gaye Hum Kahan,2023.0,tt15434074,Three best friends juggle life as 20-something...,875188.0,6.617,15.554,NaN,NaN,"['Drama', 'Comedy']","['friendship', 'coming of age', 'social media'...","['Adarsh Gourav', 'Anya Singh', 'Malaika Arora']",Arjun Varain Singh,https://media.themoviedb.org/t/p/w600_and_h900...


In [4]:
df['Tags'] = df['Overview'].apply(lambda x: x if isinstance(x, list) else [x]) + \
             df['Genre'].apply(lambda x: x if isinstance(x, list) else [x]) + \
             df['Keywords'].apply(lambda x: x if isinstance(x, list) else [x]) + \
             df['Cast'].apply(lambda x: x if isinstance(x, list) else [x]) + \
             df['Director'].apply(lambda x: x if isinstance(x, list) else [x])


In [5]:
new_df=df[["TMDB_Id","Title","Tags"]]

In [ ]:
new_df=new_df.dropna()

In [ ]:
text=new_df.iloc[459,2]
text

In [ ]:
def process_text(text):
    # Initialize stop words and stemmer
    text=str(text)
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()

    # Convert to lowercase
    text = text.lower()

    # Replace punctuation with spaces
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))

    # Split text into words
    words = text.split()

    # Stem words and remove stop words
    processed_words = [ps.stem(word) for word in words if ps.stem(word) not in stop_words]

    # Join words back into a single string
    processed_text = " ".join(processed_words)
    
    return processed_text


In [ ]:
new_df["Processed_Tags"]=new_df["Tags"].apply(process_text)

In [ ]:
cv=TfidfVectorizer(max_features=100)

In [ ]:
vectors=cv.fit_transform(new_df["Processed_Tags"]).toarray()

In [ ]:
cv.get_feature_names_out()

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity.shape

In [ ]:
def recommend(movie):
    index = new_df[new_df['Title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].Title)

In [ ]:
new_df=new_df[["TMDB_Id","Title","Processed_Tags"]]

In [ ]:
new_df

In [ ]:
recommend("Kadak Singh")

NameError: name 'recommend' is not defined

In [ ]:
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))